In [1]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.append("./SimpleITK/")

In [4]:
import matplotlib.pyplot as plt
import SimpleITK as sitk
import cv2
import numpy as np
from glob import glob
import skimage.io as io

def read_image_into_ndArray(imagefile, PlugIn):
    imageArray = io.imread(imagefile, plugin=PlugIn)
    print("The dimension of the image is ", imageArray.shape)
    return imageArray

def sitk_tile_vec(lstImgs):
    lstImgToCompose = []
    for idxComp in range(lstImgs[0].GetNumberOfComponentsPerPixel()):
        lstImgToTile = []
        for img in lstImgs:
            lstImgToTile.append(sitk.VectorIndexSelectionCast(img, idxComp))
        lstImgToCompose.append(sitk.Tile(lstImgToTile, (len(lstImgs), 1, 0)))
    sitk_show(sitk.Compose(lstImgToCompose))

def sitk_show(img, title=None, margin=0.0, dpi=40):
    nda = sitk.GetArrayFromImage(img)
    #spacing = img.GetSpacing()
    figsize = (1 + margin) * nda.shape[0] / dpi, (1 + margin) * nda.shape[1] / dpi
    #extent = (0, nda.shape[1]*spacing[1], nda.shape[0]*spacing[0], 0)
    extent = (0, nda.shape[1], nda.shape[0], 0)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

    plt.set_cmap("gray")
    ax.imshow(nda,extent=extent,interpolation=None)
    
    if title:
        plt.title(title)
    
    plt.show()
    
def figure_images(imageList, z):    
    i = 1
    for a_file in imageList:
        sliced = sitk.GetArrayViewFromImage(a_file[1])[z,:,:]        
        #plt.subplot(round(len(imageList)/3)+1,3, i)
        plt.figure()
        plt.title(a_file[0])
        plt.imshow(sliced, cmap=plt.cm.Greys_r)
        plt.axis('off')
        i = i+1

In [5]:
import SimpleITK as sitk
import skimage.io as io

## 2D images (jpg, png, etc)

In [9]:
imgfile = "data/T1.jpg"
img2 = read_image_into_ndArray(imgfile, PlugIn='matplotlib')

The dimension of the image is  (417, 428, 3)


## ITK metaImage

In [10]:
mhafile = "data/VSD.Brain.XX.O.MR_Flair.54512.mha"
img = read_image_into_ndArray(mhafile, PlugIn='simpleitk')

The dimension of the image is  (155, 240, 240)


In [11]:
mhafile2 = "data/VSD.Brain.XX.O.MR_T1.54513.mha"
img2 = read_image_into_ndArray(mhafile2, PlugIn='simpleitk')

The dimension of the image is  (155, 240, 240)


#### Multiple files

In [12]:
mhafiles = glob('mnt/hgfs/Medical-Data/BRATS-2015/BRATS2015_Training/HGG/*/*/*mha')

In [13]:
imgList = []
for a_file in mhafiles:
    filename = a_file.split("/")[-1]
    imgInput = sitk.ReadImage(a_file)
    nda = sitk.GetArrayViewFromImage(imgInput) # Get a view of the image data as a numpy array, useful for display
    img = sitk.GetImageFromArray(nda)
    imgList.append((filename, img))
    

In [14]:
figure_images(imgList, 80)

### T1 and T2-weighted image

In [16]:
from ipywidgets import interact, FloatSlider

In [17]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive
from ipywidgets import widgets

def myshow(img, title=[], margin=0.05, dpi=80 ):
    nda = sitk.GetArrayViewFromImage(img)

    spacing = img.GetSpacing()
    slicer = False

    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]

        # the the number of components is 3 or 4 consider it an RGB image
        if not c in (3,4):
            slicer = True

    elif nda.ndim == 4:
        c = nda.shape[-1]

        if not c in (3,4):
            raise Runtime("Unable to show 3D-vector Image")

        # take a z-slice
        slicer = True
    

    if (slicer):
        ysize = nda.shape[1]
        xsize = nda.shape[2]
    else:
        ysize = nda.shape[0]
        xsize = nda.shape[1]


    # Make a figure big enough to accomodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi
    def callback(z=None):        

        extent = (0, xsize*spacing[1], ysize*spacing[0], 0)

        fig = plt.figure(figsize=figsize, dpi=dpi)

        # Make the axis the right size...
        ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

        plt.set_cmap("gray")

        if z is None:
            ax.imshow(nda,extent=extent,interpolation=None)
        else:
            ax.imshow(nda[z,...],extent=extent,interpolation=None)
            if title:
                plt.title(title[z])
            
        plt.show()

    if slicer:
        if title:
            interact(callback, z=(0,nda.shape[0]-1))
        else:
            interact(callback, z=(0,nda.shape[0]-1))
    else:
        callback()

def myshow3d(img, xslices=[], yslices=[], zslices=[], title=[], margin=0.05, dpi=80):
    size = img.GetSize()
    img_xslices = [img[s,:,:] for s in xslices]
    img_yslices = [img[:,s,:] for s in yslices]
    img_zslices = [img[:,:,s] for s in zslices]

    maxlen = max(len(img_xslices), len(img_yslices), len(img_zslices))


    img_null = sitk.Image([0,0], img.GetPixelID(), img.GetNumberOfComponentsPerPixel())

    img_slices = []
    d = 0

    if len(img_xslices):
        img_slices += img_xslices + [img_null]*(maxlen-len(img_xslices))
        d += 1

    if len(img_yslices):
        img_slices += img_yslices + [img_null]*(maxlen-len(img_yslices))
        d += 1

    if len(img_zslices):
        img_slices += img_zslices + [img_null]*(maxlen-len(img_zslices))
        d +=1

    if maxlen != 0:
        if img.GetNumberOfComponentsPerPixel() == 1:
            img = sitk.Tile(img_slices, [maxlen,d])
        #TODO check in code to get Tile Filter working with VectorImages
        else:
            img_comps = []
            for i in range(0,img.GetNumberOfComponentsPerPixel()):
                img_slices_c = [sitk.VectorIndexSelectionCast(s, i) for s in img_slices]
                img_comps.append(sitk.Tile(img_slices_c, [maxlen,d]))
            img = sitk.Compose(img_comps)
    


    myshow(img, title, margin, dpi)


In [18]:
img_T1 = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T1.54513.mha")
img_T2 = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T2.54515.mha")
img_FLAIR = sitk.ReadImage("data/VSD.Brain.XX.O.MR_Flair.54512.mha")
img_T1c = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T1c.54514.mha")
# To visualize the labels image in RGBVSD.Brain.XX.O.MR_Flair.54512.mha with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)
img_T2_255 = sitk.Cast(sitk.RescaleIntensity(img_T2), sitk.sitkUInt8)
img_FLAIR_255 = sitk.Cast(sitk.RescaleIntensity(img_FLAIR), sitk.sitkUInt8)
img_T1c_255 = sitk.Cast(sitk.RescaleIntensity(img_T1c), sitk.sitkUInt8)
myshow3d(img_T1)
myshow3d(img_T1_255)

A Jupyter Widget

A Jupyter Widget

In [19]:
img_T1 = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T1.54513.mha")
img_T2 = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T2.54515.mha")
img_FLAIR = sitk.ReadImage("data/VSD.Brain.XX.O.MR_Flair.54512.mha")
img_T1c = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T1c.54514.mha")


    # 모델을 생성한다.
img_T1_Slice = img_T1[:,:,77]
img_T1c_Slice = img_T1c[:,:,77]
img_T2_Slice = img_T2[:,:,77]
img_FLAIR_Slice = img_FLAIR[:,:,77]
#imgVolume = sitk.JoinSeries(img_T1_Slice, img_T1c_Slice, img_T2_Slice, img_FLAIR_Slice)
imgVolume = sitk.JoinSeries([img_T1_Slice, img_T1c_Slice, img_T2_Slice, img_FLAIR_Slice])
width = imgVolume.GetWidth()
height = imgVolume.GetHeight()
depth = imgVolume.GetDepth()
myshow3d(imgVolume, title=['T1', 'T1c', 'T2', 'FLAIR'])

A Jupyter Widget

In [20]:
Batch_dir = "data/brats_2013_pat0014_1"
File_type = "mha"
Modality = ['T1.', 'T1c.', 'T2.', 'Flair.']
img_dir = []
for modal in Modality:
    img_dir = img_dir + glob("%s/*%s*/*%s" %(Batch_dir, modal, File_type))
Slice_num = 77
imgList = []
imgSliceList = []
for imgFile in img_dir:
    img = sitk.ReadImage(imgFile)
    imgList.append(img)
    imgSliceList.append(img[:,:,Slice_num])
imgVolume = sitk.JoinSeries(imgSliceList)
myshow3d(imgVolume, title=Modality)

A Jupyter Widget

## Synchronized Multi-modal images

In [21]:
img_T1 = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T1.54513.mha")
img_T2 = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T2.54515.mha")
img_FLAIR = sitk.ReadImage("data/VSD.Brain.XX.O.MR_Flair.54512.mha")
img_T1c = sitk.ReadImage("data/VSD.Brain.XX.O.MR_T1c.54514.mha")
img_GT = sitk.ReadImage("data/VSD.Brain_3more.XX.O.OT.54517.mha")

In [22]:
nda1 = sitk.GetArrayViewFromImage(img_T1)
nda2 = sitk.GetArrayViewFromImage(img_T2)
nda3 = sitk.GetArrayViewFromImage(img_T1c)
nda4 = sitk.GetArrayViewFromImage(img_FLAIR)
nda5 = sitk.GetArrayViewFromImage(img_GT)
spacing = img_T1.GetSpacing()
slicer = False
title = []
ysize= nda1.shape[1]
xsize= nda1.shape[2]
margin=0.05
dpi=80
figsize = 4*(1 + margin) * ysize / dpi, 4*(1 + margin) *xsize / dpi
print(figsize)
print(nda1.shape)

(12.6, 12.6)
(155, 240, 240)


In [23]:
def callback(z=None):        

    extent = (0, xsize*spacing[1], ysize*spacing[0], 0)
    fig = plt.figure(figsize=figsize, dpi=dpi)      
    plt.subplot(151)
    plt.title("T1")
    plt.set_cmap("gray")
    plt.imshow(nda1[z,...],extent=extent,interpolation=None)    
    plt.subplot(152)
    plt.title("T2")
    plt.set_cmap("gray")
    plt.imshow(nda2[z,...],extent=extent,interpolation=None)
    plt.subplot(153)
    plt.title("T1c")
    plt.set_cmap("gray")
    plt.imshow(nda3[z,...],extent=extent,interpolation=None)
    plt.subplot(154)
    plt.title("FLAIR")
    plt.set_cmap("gray")
    plt.imshow(nda4[z,...],extent=extent,interpolation=None)
    plt.subplot(155)
    plt.title("Ground Truth")
    plt.set_cmap("gray")
    plt.imshow(nda5[z,...],extent=extent,interpolation=None)
    
       
    plt.show()

In [24]:
interact(callback, z=(0,nda1.shape[0]-1))

A Jupyter Widget

<function __main__.callback>